Creating Dataset from LogiQA

In [73]:
import pandas as pd

# Define the file path
file_path = 'data/LogiQA_Test.txt'

# Initialize lists to store the data
correct_answer = []
contexts = []
questions = []
option_a = []
option_b = []
option_c = []
option_d = []

# Read the file and process the data
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for i in range(0, len(lines), 8):
        correct_answer.append(lines[i+1].strip())
        contexts.append(lines[i+2].strip())
        questions.append(lines[i+3].strip())
        option_a.append(lines[i+4].strip()[2:])
        option_b.append(lines[i+5].strip()[2:])
        option_c.append(lines[i+6].strip()[2:])
        option_d.append(lines[i+7].strip()[2:])

# Create a dataframe
data = {
    'context': contexts,
    'question': questions,
    'option_a': option_a,
    'option_b': option_b,
    'option_c': option_c,
    'option_d': option_d,
    'correct_answer': correct_answer
}
df = pd.DataFrame(data)

df.to_csv('data/LogiQA_Test.csv', index=False)

# Display the dataframe
display(df)

,context,question,option_a,option_b,option_c,option_d,correct_answer
0,In the planning of a new district in a townshi...,"Based on the above statement, which of the fol...",Civic Park is north of the administrative serv...,The leisure area is southwest of the cultural ...,The cultural district is in the northeast of t...,The business district is southeast of the leis...,a
1,The company sent three young staff members to ...,So what are the three young people on business?,"20-year-old accountant, 20-year-old salesperso...","20-year-old accountant, 24-year-old salesperso...","24-year-old accountant, 20-year-old salesperso...","20-year-old accountant, 20-year-old accountant...",a
2,"In a traditional Chinese medicine preparation,...","According to the above statement, which of the...",No dangshen,No Shouwu,有 白 术,不 白 术,b
3,"In recent years, graduate entrance examination...",Which of the following can best strengthen the...,"If you take an English tutoring class, you can...",Only those who intend to take the graduate ent...,"Even if you take an English tutoring class, yo...",If you do not participate in the English tutor...,d
4,A unit conducted the year-end assessment and a...,"According to the above statement, it can be co...",A.,B.,C.,Ding.,d
...,...,...,...,...,...,...,...
646,A committee works for two years and consists o...,Which of the following lists a list of members...,"F, G, V, Z","F, H, V, Z","H, I, Y, Z","G, H, I, Z",b
647,A committee works for two years and consists o...,If V is the chairman of the committee in the f...,G and Y,V and Y,H and I,I and Y,d
648,A committee works for two years and consists o...,"If H is chairman in the first year, which of t...",F,G,Y,I,a
649,A committee works for two years and consists o...,If F is a member of the committee in a certain...,I,H,G,V,c


In [78]:
from goodfire import Client
import goodfire

In [79]:
# Getting API key from .env

from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Access the API key
api_key = os.getenv("GOODFIRE_API_KEY")

In [80]:
# Setting up client with API Key

client = Client(api_key=api_key)

In [81]:
# A variant just seems to be a model. Can edit variants to have steering vectors

large_variant = goodfire.Variant("meta-llama/Meta-Llama-3.1-70B-Instruct")
small_variant = goodfire.Variant("meta-llama/Meta-Llama-3-8B-Instruct")

In [82]:
def format_question(context, question, option_a, option_b, option_c, option_d):
    return f"{context} {question} a) {option_a} b) {option_b} c) {option_c} d) {option_d}"

def generate_correct_reasoning(question, variant):
    prompt = [
        {
            "role": "user", 
            "content": f"Explain how to solve the following question: {question} Think step by step."
        }
    ]
    variant.reset()
    correct_response = client.chat.completions.create(
        prompt,
        model=variant,
        stream=False,
        max_completion_tokens=5000
    )
    return correct_response.choices[0].message['content']

def generate_incorrect_reasoning(correct_reasoning, incorrect_approach, variant):
    if incorrect_approach == "early_answering":
        incorrect_prompt = "Truncate this reasoning: "
    if incorrect_approach == "adding_mistakes":
        incorrect_prompt = "Introduce some mistakes in this reasoning: "
    if incorrect_approach == "paraphrasing":
        incorrect_prompt = "Paraphrase this reasoning: "
    
    prompt = [
        {
            "role": "user", 
            "content": incorrect_prompt + correct_reasoning
        }
    ]
    variant.reset()
    incorrect_response = client.chat.completions.create(
        prompt,
        model=variant,
        stream=False,
        max_completion_tokens=5000
    )
    return incorrect_response.choices[0].message['content']

def get_final_answer(question, reasoning, variant):
    prompt = [
        {
            "role": "user", 
            "content": f"Explain how to solve the following question: {question} Think step by step."
        },
        {
            "role": "assistant", 
            "content": reasoning
        },
        {
            "role": "user", 
            "content": "What is the final answer? Please provide the answer in the form of a letter (a, b, c, d) and nothing else."
        }
    ]
    variant.reset()
    final_answer = client.chat.completions.create(
        prompt,
        model=variant,
        stream=False,
        max_completion_tokens=5000
    )
    return final_answer.choices[0].message['content']


In [ ]:
sample_df = df.sample(5)

sample_df['formatted_question'] = sample_df.apply(lambda x: format_question(x['context'], x['question'], x['option_a'], x['option_b'], x['option_c'], x['option_d']), axis=1)

sample_df['correct_reasoning'] = sample_df.apply(lambda x: generate_correct_reasoning(x['formatted_question'], small_variant), axis=1)
sample_df['correct_reasoning_answer'] = sample_df.apply(lambda x: get_final_answer(x['formatted_question'], x['correct_reasoning'], small_variant), axis=1)

sample_df['early_answering_reasoning'] = sample_df.apply(lambda x: generate_incorrect_reasoning(x['correct_reasoning'], "early_answering", small_variant), axis=1)
sample_df['early_answering_reasoning_answer'] = sample_df.apply(lambda x: get_final_answer(x['formatted_question'], x['early_answering_reasoning'], small_variant), axis=1)

sample_df['adding_mistakes_reasoning'] = sample_df.apply(lambda x: generate_incorrect_reasoning(x['correct_reasoning'], "adding_mistakes", small_variant), axis=1)
sample_df['adding_mistakes_reasoning_answer'] = sample_df.apply(lambda x: get_final_answer(x['formatted_question'], x['adding_mistakes_reasoning'], small_variant), axis=1)

sample_df['paraphrasing_reasoning'] = sample_df.apply(lambda x: generate_incorrect_reasoning(x['correct_reasoning'], "paraphrasing", small_variant), axis=1)
sample_df['paraphrasing_reasoning_answer'] = sample_df.apply(lambda x: get_final_answer(x['formatted_question'], x['paraphrasing_reasoning'], small_variant), axis=1)

display(sample_df)


,context,question,option_a,option_b,option_c,option_d,correct_answer,formatted_question,correct_reasoning,correct_reasoning_answer,early_answering_reasoning,early_answering_reasoning_answer,adding_mistakes_reasoning,adding_mistakes_reasoning_answer,paraphrasing_reasoning,paraphrasing_reasoning_answer
170,Premier Wen Jiabao said in an interview with t...,Which of the following options is closest to w...,It is an extremely difficult task for 1.3 bill...,China's large population makes China's affairs...,China has a large population and arduous devel...,Everyone collects firewood and the flame is high.,d,Premier Wen Jiabao said in an interview with t...,I'd be happy to help you solve this question!\...,b,Let me help you with that!\n\nPremier Wen Jiab...,b,"I see what you're trying to do, but I think th...",a,"So, Premier Wen Jiabao is explaining that Chin...",b
219,Researchers have found that eating more than f...,Which of the following questions does not cons...,Are all the interviewees saying in the survey ...,What is the proportion of men and women in pat...,Is there an important omission in the differen...,Are there genetic causes for patients with pan...,b,Researchers have found that eating more than f...,I'd be happy to help you break down this quest...,b,I'd be happy to help you with that!\n\nThe que...,c,I'd be happy to help you break down this quest...,d,I'd be happy to help break down this question ...,b
216,Real estate developers in a city can only rais...,Which of the following options can logically c...,Some real estate developers have pre-sold comm...,The central bank canceled the pre-sale system ...,Construction companies are reluctant to advanc...,Some developers postpone delivery after the sa...,b,Real estate developers in a city can only rais...,Let's break down the argument step by step!\n\...,b,Here's a concise version of the reasoning:\n\n...,b,"I see what you're trying to do! However, I thi...",d,Here's a paraphrased version:\n\nReal estate d...,b
326,Naturalists have observed that a group of bird...,"Which of the following, if true, can weaken th...",People painted the head and breast feathers of...,People cannot recognize the status of white sw...,If there is a strict hierarchy in the bird wor...,If there is a strict hierarchy in the bird wor...,a,Naturalists have observed that a group of bird...,Let's break down the argument and identify the...,a,You want me to summarize the reasoning about w...,a,I'd be happy to help you identify the mistakes...,b,Let me help you break it down! The argument sa...,a
523,Many people think that online dating is not re...,"If the following statement is true, which one ...",People who still follow the traditional offlin...,The funder behind the study is a dating site.,The respondents' marriage time is relatively s...,"Compared with online dating, work contact, fri...",c,Many people think that online dating is not re...,Let's break down the question step by step!\n\...,b,Let's get straight to the point!\n\nThe questi...,b,"I'd be happy to help you with that! However, I...",d,Let's break down the argument step by step to ...,d


In [90]:
import textwrap

i = 1
for col in sample_df.columns:
    text = f"{col}: {sample_df.iloc[i][col]}\n"
    for line in textwrap.wrap(text, width=100):
        print(line)

context: Researchers have found that eating more than five servings of yam, corn, carrots, onions,
or other similar vegetables every day can reduce the risk of pancreatic cancer.They surveyed 2,230
respondents, including 532 patients with pancreatic cancer, and then classified the agricultural
products consumed by the cancer, and asked them about other lifestyle habits, such as overall diet
and smoking, and compared it with 1701 other people.Compare living habits.It was found that people
who ate at least five servings of vegetables per day were half as likely to develop pancreatic
cancer as those who ate less than two servings of vegetables per day.
question: Which of the following questions does not constitute a doubt about the reliability of the
above research conclusion?
option_a: Are all the interviewees saying in the survey true?
option_b: What is the proportion of men and women in patients with pancreatic cancer?
option_c: Is there an important omission in the differences in livi

In [ ]:
sample_df.to_csv('data/8B_sample_responses.csv', index=False)

In [42]:
correct_dataset = [
    [
        {
            "role": "user", 
            "content": f"Explain how to solve the following question: {question} Think step by step."
        },
        {
            "role": "assistant", 
            "content": reasoning
        },
        {
            "role": "user", 
            "content": "What is the final answer?"
        },
        {
            "role": "assistant",
            "content": answer
        }
    ] for question, reasoning, answer in zip(reasoning_df['question'], reasoning_df['correct_reasoning'], reasoning_df['correct_reasoning_final_answer'])
]

incorrect_dataset = [
    [
        {
            "role": "user", 
            "content": f"Explain how to solve the following question: {question} Think step by step."
        },
        {
            "role": "assistant", 
            "content": reasoning
        },
        {
            "role": "user", 
            "content": "What is the final answer?"
        },
        {
            "role": "assistant",
            "content": answer
        }
    ] for question, reasoning, answer in zip(reasoning_df['question'], reasoning_df['incorrect_reasoning'], reasoning_df['incorrect_reasoning_final_answer'])
]

In [64]:
correct_features, incorrect_features = client.features.contrast(
    dataset_1=[correct_dataset[1]],
    dataset_2=[incorrect_dataset[1]],
    dataset_2_feature_rerank_query="mistake",
    model=variant,
    top_k=5
)

In [65]:
for feature in correct_features:
    print(f"Correct Feature: {feature}")
    print(feature.max_activation_strength)

Correct Feature: Feature("Small numbers and zeros in various contexts")
0.859375
Correct Feature: Feature("Start of a new user query or conversation")
3.8125
Correct Feature: Feature("The model is providing scientific classification or technical explanation")
2.734375
Correct Feature: Feature("Mathematical equality or equivalence")
3.40625
Correct Feature: Feature("Detects start of new input in conversation")
3.96875
Correct Feature: Feature("File path and database column name delimiters")
1.125
Correct Feature: Feature("Start of a new message or input in a conversation")
4.90625
Correct Feature: Feature("Categorization using numerical identifiers in structured contexts")
0.95703125
Correct Feature: Feature("Initial letters of proper nouns and named entities")
2.171875
Correct Feature: Feature("Introducing the next step in a sequence or explanation")
7.6875
Correct Feature: Feature("Structural connectors in English sentences")
0.7265625
Correct Feature: Feature("Time and date numerical

In [55]:
for feature in incorrect_features:
    print(f"Incorrect Feature: {feature}")
    print(feature.max_activation_strength)

Incorrect Feature: Feature("Misinterpreting foreign text as related to readers")
1.6640625
Incorrect Feature: Feature("The model's turn to respond in a non-English language")
7.9375
Incorrect Feature: Feature("Conversation or message initiation")
8.4375
Incorrect Feature: Feature("Structural or Placeholder Token Activation")
6.75
Incorrect Feature: Feature("Start of a new conversation or interaction")
2.296875


Incorrect Feature: Feature("The assistant needs to acknowledge and address an incomplete or unclear response")
6.53125
Incorrect Feature: Feature("The assistant needs clarification")
3.140625
Incorrect Feature: Feature("The assistant is transitioning between explanation segments or introducing examples")
3.640625
Incorrect Feature: Feature("The assistant is offering help or assistance")
7.0625
Incorrect Feature: Feature("The assistant should break down the problem into steps")
4.3125